### Preamble

In [43]:
import ROOT, rootpy
from ROOT import gSystem
import root_numpy as rnp
import numpy as np
from dependencies import root_dependencies
root_dependencies()

Load files

In [3]:
numuCC_files = !ls /home/cdesio/trigger_optimisation/neutrinos/numuCC/out_JTE/*.JTE.root

In [4]:
fname = numuCC_files[30]

In [20]:
fname

'/home/cdesio/trigger_optimisation/neutrinos/numuCC/out_JTE/km3_v4_numucc_1.evt.JTE.root'

In [8]:
montecarlo_energies = rnp.root2array(fname, treename="MONTECARLO", branches="neutrino_.E_")

Retrieve information about number of events and event number of the MC events to create the histo`

In [9]:
eventnumber = rnp.root2array(fname, treename='MONTECARLO', branches='eventNumber_').astype(np.int)
max_evn = np.int(np.max(eventnumber))
ndoms = 2070

In [10]:
eventnumber

array([   1,    2,    8, ..., 6304, 6305, 6306])

In [11]:
import rootpy.plotting
from rootpy.io import root_open

In [12]:
root_file = root_open(fname)
rpy_tree = root_file.MONTECARLO

Create the 2D histogram to extract the number of hit DOMs per event

In [13]:
h2d = rootpy.plotting.Hist2D(np.int(np.max(eventnumber)), np.min(eventnumber), np.max(eventnumber), ndoms, 0, ndoms, type="F")

In [14]:
ret = rpy_tree.Draw('eventNumber_:HitList_.pm_id_/31', hist=h2d, create_hist=False)

Extraction of the number of hit doms from the histo:
the for loop on x is done on the eventnumber array, in order to take only the ids of the events that are contained in the array;
the for loop on y is done on the number of doms;
after the two loops, the number of hit doms per event is written in the output array, which has the same length of the mc array

In [15]:
ev_countdoms = np.zeros(eventnumber.shape)
for i, x in enumerate(eventnumber):
    count = 0
    ix = np.int(x)
    for y in range(ndoms):
        if h2d.GetBinContent(ix,y) > 0:
            count+=1
    if count > 0:
        ev_countdoms[i] = count

Select the events with number of doms > 5

In [16]:
selected = ev_countdoms[ev_countdoms>5]

Get the index of the events with number of hit doms >5

In [17]:
sel_idx = np.where(ev_countdoms>5)[0]

In [18]:
sel_idx

array([   0,    3,    9, ..., 3849, 3850, 3851])

Show that all the selected events have number of doms >5

In [19]:
np.all(ev_countdoms[sel_idx]>5)

True

Further check: compare the number of hits with the number of hit doms, to prove that the latter is <= than the first

In [32]:
montecarlo_numberofhits = rnp.root2array(fname, treename="MONTECARLO", branches="@HitList_.size()").astype(np.int)

Let's see it

In [165]:
montecarlo_numberofhits[:10]

array([15,  1,  1, 16,  2,  3,  5,  9,  1, 45])

In [166]:
ev_countdoms[:10]

array([  7.,   1.,   1.,  12.,   2.,   3.,   5.,   4.,   1.,  15.])

Numerical evaluation

In [168]:
np.all(ev_countdoms<=montecarlo_numberofhits)

True

Select the energies of the mc events with more than 5 doms

In [169]:
sel_energies = montecarlo_energies[sel_idx]

In [170]:
sel_energies

array([  1.21790000e+02,   1.12132000e+02,   1.97317000e+02, ...,
         2.88259000e+07,   5.26814000e+07,   2.52789000e+07])

Evaluate the number of events per energy bin with np.histogram

In [4]:
binned_selected = np.histogram(np.log10(sel_energies),bins = 50, range=(2,8))

NameError: name 'np' is not defined

In [178]:
binned

(array([ 41,  38,  61,  54,  56,  66,  78,  77,  89,  70,  89,  83,  98,
         93,  77,  72, 103,  92,  82,  98,  88,  77,  69,  79,  67,  69,
         88,  62,  54,  46,  62,  53,  36,  43,  25,  36,  32,  28,  23,
         25,  23,  21,  23,  26,  22,  13,  14,  11,  11,  11]),
 array([ 2.  ,  2.12,  2.24,  2.36,  2.48,  2.6 ,  2.72,  2.84,  2.96,
         3.08,  3.2 ,  3.32,  3.44,  3.56,  3.68,  3.8 ,  3.92,  4.04,
         4.16,  4.28,  4.4 ,  4.52,  4.64,  4.76,  4.88,  5.  ,  5.12,
         5.24,  5.36,  5.48,  5.6 ,  5.72,  5.84,  5.96,  6.08,  6.2 ,
         6.32,  6.44,  6.56,  6.68,  6.8 ,  6.92,  7.04,  7.16,  7.28,
         7.4 ,  7.52,  7.64,  7.76,  7.88,  8.  ]))

In [184]:
binned_mc = np.histogram(np.log10(montecarlo_energies), bins=50, range=(2,8))

In [185]:
eff = np.true_divide(binned_selected[0], binned_mc[0])

In [186]:
eff

array([ 0.36607143,  0.36893204,  0.49193548,  0.51428571,  0.53333333,
        0.61111111,  0.62903226,  0.66956522,  0.7007874 ,  0.67961165,
        0.712     ,  0.65873016,  0.72592593,  0.7265625 ,  0.74757282,
        0.6728972 ,  0.79844961,  0.73015873,  0.76635514,  0.79032258,
        0.75862069,  0.82795699,  0.76666667,  0.7745098 ,  0.77011494,
        0.81176471,  0.90721649,  0.86111111,  0.83076923,  0.71875   ,
        0.82666667,  0.85483871,  0.85714286,  0.87755102,  0.75757576,
        0.8372093 ,  0.7804878 ,  0.8       ,  0.6969697 ,  0.75757576,
        0.74193548,  0.80769231,  0.85185185,  0.92857143,  1.        ,
        0.86666667,  0.875     ,  1.        ,  0.91666667,  0.91666667])

In [189]:
from bokeh.plotting import figure, show, output_notebook

In [3]:
colors_list = ['red']

legend_list = ['numuCC_st']

xs = binned[0]


output_notebook()
p = figure(title="Trigger efficiency", plot_height=450, plot_width=850)
p.xaxis.axis_label = "Log Energy (GeV)"
p.yaxis.axis_label = "Triggered/MC event ratio"

for (colr, leg, x, y) in zip(colors_list, legend_list, xs, ys):
    myplot = p.line(x,y,color=colr, legend=leg, line_width=2)
p.legend.location = "bottom_right"
show(p)

NameError: name 'binned' is not defined

Check on the histogram binning

In [128]:
with open('underflow_overflow.txt','w') as f:
    for x in range(0,numberOfEvents+1):
        for y in range(0, ndoms+1):
            if h2d.IsBinUnderflow(h2d.GetBin(x,y)):
                f.write('underderflow {} {}\n'.format( x, y))
            elif h2d.IsBinOverflow(h2d.GetBin(x,y)):
                f.write('overflow {} {}\n'.format(x, y))

In [1]:
from root_pandas import read_root

In [7]:
df = read_root(fname, 'MONTECARLO', columns='HitList_.pm_id_')

ValueError: Pattern 'HitList_.pm_id_' didn't match any branch

In [8]:
from root_numpy import list_branches

In [9]:
branches = list_branches(fname, 'MONTECARLO')

In [10]:
branches

['MONTECARLO']

In [11]:
import root_numpy

In [15]:
root_numpy.list_branches(fname, root_numpy.list_trees(fname)[1])

['MONTECARLO']

In [20]:
root_numpy.list_structures(fname,'MONTECARLO')

OrderedDict([('MONTECARLO', [('MONTECARLO', u'Event')])])

In [28]:
import root_pandas

In [34]:
root_pandas.read_root(fname,'MONTECARLO.HitList_.pm_id_')

IOError: tree 'MONTECARLO.HitList_.pm_id_' not found in /home/cdesio/trigger_optimisation/neutrinos/numuCC/out_JTE/km3_v4_numucc_1.evt.JTE.root

In [37]:
from rootpy import root2hdf5

In [50]:
import tables

In [51]:
out = tables.file

In [52]:
root2hdf5.root2hdf5(fname, out)

AttributeError: 'module' object has no attribute 'root'